In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from global_import import *
from valuation.val import perform_valuation
from fm import fama_macbeth_reg_panel_nw

In [2]:
ret_decomp_dcf5 = \
          pd.read_csv(f'{DATA_FOLDER}/decomp_dcf5y_RF_rolling_1980_2019.csv', index_col=0).replace([np.inf, -np.inf], np.nan)
ret_decomp_dcf5 = ret_decomp_dcf5[ret_decomp_dcf5.prc>5]

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
ret_decomp_dcf5.head(3)

,permno,ticker,exchcd,year,jdate,exchcd.1,vt,me,be,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_48,logret_60,cumret_12,cumret_36,cumret_48,cumret_60,vt+3,me+3,be+3,vt+6,me+6,be+6,vt+12,me+12,be+12,vt+24,me+24,be+24,vt+36,me+36,be+36,vt+48,me+48,be+48,vt+60,me+60,be+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+48,r*_t+48,r_unchg+48,r_chg+48,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp
287,10001,EGAS,2,2014,2014-12-31,2,9.083331,11.657699,106669.0,-0.087672,-0.054105,0.222733,-0.083938,-0.052667,0.249487,-0.331516,-0.165001,-0.165001,-0.165001,-0.282165,-0.152107,-0.152107,-0.152107,8.553669,11.553126,106962.0,8.553669,11.590608,106962.0,8.553669,11.267821,106962.0,6.240276,11.790754,107784.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.087672,-0.529662,-0.064359,0.414462,-0.054105,-0.529662,-0.064359,0.451007,-0.331516,-0.529662,-0.064359,0.136289,0.222733,-2.843055,-0.064359,2.901708,-0.165001,NaN,-0.064359,NaN,-0.165001,NaN,-0.064359,NaN,-0.165001,NaN,-0.064359,NaN,2.574368
286,10001,EGAS,2,2014,2014-11-30,2,9.083331,11.700334,106669.0,-0.125736,-0.122224,0.174223,-0.118153,-0.115050,0.190321,-0.228339,-0.203561,-0.203561,-0.203561,-0.204145,-0.184179,-0.184179,-0.184179,8.553669,11.570522,106962.0,8.553669,11.560572,106962.0,8.553669,11.427517,106962.0,6.240276,11.786762,107784.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.125736,-0.529662,-0.065425,0.389854,-0.122224,-0.529662,-0.065425,0.380153,-0.228339,-0.529662,-0.065425,0.250424,0.174223,-2.843055,-0.065425,2.856246,-0.203561,NaN,-0.065425,NaN,-0.203561,NaN,-0.065425,NaN,-0.203561,NaN,-0.065425,NaN,2.617003
285,10001,EGAS,2,2014,2014-10-31,2,9.083331,11.687205,106669.0,-0.143970,-0.098213,0.179185,-0.134086,-0.093545,0.196242,-0.202192,-0.186526,-0.186526,-0.186526,-0.183062,-0.170163,-0.170163,-0.170163,8.553669,11.535254,106962.0,8.553669,11.564109,106962.0,8.553669,11.436152,106962.0,6.240276,11.774690,107784.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.143970,-0.529662,-0.065097,0.368269,-0.098213,-0.529662,-0.065097,0.396403,-0.202192,-0.529662,-0.065097,0.271645,0.179185,-2.843055,-0.065097,2.857277,-0.186526,NaN,-0.065097,NaN,-0.186526,NaN,-0.065097,NaN,-0.186526,NaN,-0.065097,NaN,2.603874


In [4]:
ret_decomp_dcf5_nyse = ret_decomp_dcf5[ret_decomp_dcf5.exchcd==1]

In [5]:
ret_decomp_dcf5_nyse.groupby(pd.qcut(ret_decomp_dcf5_nyse.me, 10)).agg('mean')[['misp', 'r_t+60']]

,misp,r_t+60
me,,
"(4.662, 11.301]",-0.336157,0.320526
"(11.301, 11.956]",0.052478,0.305909
"(11.956, 12.497]",0.008589,0.335721
"(12.497, 13.015]",-0.005350,0.345394
"(13.015, 13.495]",-0.012955,0.343499
"(13.495, 13.982]",0.066399,0.345045
"(13.982, 14.499]",0.031073,0.337384
"(14.499, 15.081]",0.042420,0.320676
"(15.081, 15.949]",0.098401,0.323911


In [6]:
ret_decomp_dcf5_nyse.groupby(pd.qcut(ret_decomp_dcf5_nyse.me, 10)).agg('std')[['misp', 'r_t+60']]

,misp,r_t+60
me,,
"(4.662, 11.301]",2.218952,0.947117
"(11.301, 11.956]",1.905475,0.775311
"(11.956, 12.497]",1.880620,0.751599
"(12.497, 13.015]",1.869999,0.760086
"(13.015, 13.495]",1.777204,0.755752
"(13.495, 13.982]",1.686438,0.766326
"(13.982, 14.499]",1.683517,0.754568
"(14.499, 15.081]",1.592577,0.693894
"(15.081, 15.949]",1.540567,0.646462


In [7]:
ret_decomp_dcf5_nyse['bm'] = np.exp(np.log(ret_decomp_dcf5_nyse['be']) - ret_decomp_dcf5_nyse['me'])

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
ret_decomp_dcf5_nyse.groupby(pd.qcut(ret_decomp_dcf5_nyse.bm, 10)).agg('mean')[['misp', 'r_t+60']]

,misp,r_t+60
bm,,
"(-0.000923, 0.221]",0.436106,0.170207
"(0.221, 0.326]",0.372537,0.225126
"(0.326, 0.417]",0.277216,0.281585
"(0.417, 0.514]",0.261718,0.288721
"(0.514, 0.623]",0.207166,0.322881
"(0.623, 0.753]",0.188618,0.360191
"(0.753, 0.918]",0.159921,0.399275
"(0.918, 1.158]",0.091684,0.467252
"(1.158, 1.715]",-0.036376,0.596396


In [9]:
ret_decomp_dcf5_nyse.groupby(pd.qcut(ret_decomp_dcf5_nyse.bm, 10)).agg('std')[['misp', 'r_t+60']]

,misp,r_t+60
bm,,
"(-0.000923, 0.221]",1.469845,0.825350
"(0.221, 0.326]",1.389421,0.752594
"(0.326, 0.417]",1.457042,0.715215
"(0.417, 0.514]",1.506707,0.758186
"(0.514, 0.623]",1.542499,0.771417
"(0.623, 0.753]",1.531183,0.743067
"(0.753, 0.918]",1.548525,0.717868
"(0.918, 1.158]",1.500716,0.704114
"(1.158, 1.715]",1.516049,0.751362


In [10]:
ret_decomp_dcf5['jdate'] = pd.to_datetime(ret_decomp_dcf5.jdate)
ret_decomp_dcf5_annual = ret_decomp_dcf5[ret_decomp_dcf5.jdate.dt.month==6]


In [14]:
regdf = ret_decomp_dcf5_annual[['jdate', 'permno', 'ticker', 'me', 'misp']].replace([np.inf, -np.inf], np.nan).dropna()
print(fama_macbeth_reg_panel_nw(regdf, xname='me', yname='misp'))


           count      mean  std_error     tstat  std_error_nw  tstat_nw
alpha_i_t   35.0 -0.689979   0.153508 -4.494732      1.528893 -0.451293
me          35.0  0.063620   0.009341  6.810640      0.095312  0.667490


In [15]:
ret_decomp_dcf5_annual['bm'] = np.exp(np.log(ret_decomp_dcf5_annual['be']) - ret_decomp_dcf5_annual['me'])
regdf = ret_decomp_dcf5_annual[['jdate', 'permno', 'ticker', 'bm', 'misp']].replace([np.inf, -np.inf], np.nan).dropna()
print(fama_macbeth_reg_panel_nw(regdf, xname='bm', yname='misp'))


/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


           count      mean  std_error     tstat  std_error_nw  tstat_nw
alpha_i_t   35.0  0.168692   0.092473  1.824237      0.885120  0.190586
bm          35.0 -0.021710   0.003371 -6.439436      0.027454 -0.790756
